# Demo Script for Milestone 10.15

## 0 Prepare task

### 0.1 prepare planning scene

In [1]:
import os
import sys
sys.path.append(os.path.join(os.path.join(os.environ["RNB_PLANNING_DIR"], 'src')))

from pkg.global_config import RNB_PLANNING_DIR
from demo_utils.kiro_udp_send import start_mobile_udp_thread, send_pose_wait, get_xyzw_cur, get_reach_state
from pkg.utils.utils import *    
from pkg.utils.rotation_utils import *
from pkg.controller.combined_robot import *
from pkg.project_config import *
from demo_utils.streaming import *
from demo_utils.detect_table import *
from demo_utils.area_select import *
from pkg.detector.aruco.marker_config import get_aruco_map
aruco_map = get_aruco_map()


CONNECT_CAM = False # True
ENABLE_DETECT = False
DETECT_MARKER = True
CONNECT_INDY = False # True
CONNECT_MOBILE = False # True 
SHOW_MOTION_RVIZ = False

# Tool dimensions
TOOL_DIM = [0.34, 0.10]
TOOL_OFFSET = 0
ROBOT_Z_ANGLE = np.pi
MARGIN = 0
TRACK_THICKNESS = 0.001

INDY_BASE_OFFSET = (0.172,0,0.439)
ROBOT_Z_ANGLE = np.pi
TOOL_NAME = "brush_face"
WALL_THICKNESS = 0.01
CLEARANCE = 0.001

ip_cur =  get_ip_address()
MOBILE_IP = "192.168.0.102"
INDY_IP = "192.168.0.3"
CAM_HOST = '192.168.0.10'

print("Current PC IP: {}".format(ip_cur))
print("Mobile ROB IP: {}".format(MOBILE_IP))
print("CAM SERVER IP: {}".format(CAM_HOST))

from pkg.geometry.builder.scene_builder import SceneBuilder
from demo_utils.environment import *

sock_mobile, server_thread = start_mobile_udp_thread(recv_ip=ip_cur)

mobile_config = RobotConfig(0, RobotType.kmb, ((0,0,0), (0,0,0)),
                MOBILE_IP)
robot_config = RobotConfig(1, RobotType.indy7, (INDY_BASE_OFFSET, (0,0,np.pi)),
                INDY_IP, root_on="kmb0_platform", specs={"no_sdk":True})
MOBILE_NAME = mobile_config.get_indexed_name()
ROBOT_NAME = robot_config.get_indexed_name()
crob = CombinedRobot(robots_on_scene=[mobile_config, robot_config]
              , connection_list=[False, CONNECT_INDY])

s_builder = SceneBuilder(None)
gscene = s_builder.create_gscene(crob)

gtems = s_builder.add_robot_geometries(color=(0,1,0,0.5), display=True, collision=True)
gscene.set_workspace_boundary(-3, 7, -5, 5, -CLEARANCE, 3, thickness=WALL_THICKNESS)


from pkg.planning.scene import PlanningScene
pscene = PlanningScene(gscene, combined_robot=crob)

ROBOT_BASE = pscene.robot_chain_dict[ROBOT_NAME]['link_names'][0]
TIP_LINK = pscene.robot_chain_dict[ROBOT_NAME]["tip_link"]
MOBILE_BASE = pscene.robot_chain_dict[MOBILE_NAME]["tip_link"]
HOLD_LINK = MOBILE_BASE

viewpoint = add_cam(gscene, tool_link=TIP_LINK)
# add_indy_tool_kiro(gscene, tool_link=TIP_LINK, face_name=TOOL_NAME, zoff=TOOL_OFFSET)

HOME_POSE = -crob.home_pose
HOME_DICT = list2dict(HOME_POSE, gscene.joint_names)

from pkg.planning.pipeline import PlanningPipeline
ppline = PlanningPipeline(pscene)

# Set planner
from pkg.planning.motion.moveit.moveit_planner import MoveitPlanner
from pkg.planning.filtering.grasp_filter import GraspChecker
mplan = MoveitPlanner(pscene, enable_dual=False, incremental_constraint_motion=True)
mplan.motion_filters = [GraspChecker(pscene)]
mplan.update_gscene()
from pkg.planning.task.rrt import TaskRRT
tplan = TaskRRT(pscene)
tplan.prepare()
ppline.set_motion_planner(mplan)
ppline.set_task_planner(tplan)

from pkg.ui.ui_broker import *

# start UI
ui_broker = UIBroker.instance()
ui_broker.initialize(ppline, s_builder)
ui_broker.start_server()

ui_broker.set_tables()

add_kiro_indytool_down(gscene, zoff=TOOL_OFFSET, tool_link=TIP_LINK, face_name=TOOL_NAME)

# Register binders
from pkg.planning.constraint.constraint_actor import VacuumTool, Gripper2Tool, PlacePlane, SweepFramer, WayFramer
brush_face = pscene.create_binder(bname=TOOL_NAME, gname=TOOL_NAME, _type=SweepFramer, 
                                  point=(-gscene.NAME_DICT['brush_face'].dims[0]/2-CLEARANCE,0,0), 
                                  rpy=(0,np.pi/2*1,0))

# waypoint
WP_DIMS = (0.6,0.4,WALL_THICKNESS)
gscene.create_safe(gtype=GEOTYPE.BOX, name="wayframer", link_name=HOLD_LINK,
                   dims=WP_DIMS, center=(0,0,WP_DIMS[2]/2), rpy=(0,0,0), color=(1, 0, 0, 0.5), display=True,
                   collision=False, fixed=True)
wayframer = pscene.create_binder(bname="wayframer", gname="wayframer", _type=WayFramer, 
                                 point=(0,0,-WP_DIMS[2]/2-CLEARANCE), rpy=(0,0,0))

Current PC IP: 192.168.0.123
Mobile ROB IP: 192.168.0.102
CAM SERVER IP: 192.168.0.10
[MOBILE ROBOT] bind: ('192.168.0.123', 50306)
[MOBILE ROBOT] Start UDP THREAD
connection command:
kmb0: False
indy1: False
Unable to register with master node [http://localhost:11311]: master may not be running yet. Will keep trying.
Please create a subscriber to the marker
publication OK
published: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Please create a subscriber to the marker
Dash is running on http://0.0.0.0:8050/

 * Serving Flask app "pkg.ui.dash_launcher" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


### 0.2 Wait task start queue

## 1. Bed cleaning

### 1.1 Detect bed

#### 1.1.1 Move to bed-seek pose 

In [2]:
VIEW_POSE = np.deg2rad([  0., -50.,  70.,  -0.,  90., -180])
VIEW_LOC = [0,]*6
VIEW_POSE_EXT = np.array(VIEW_LOC + list(VIEW_POSE))
if CONNECT_INDY:
    with indy:
        indy.joint_move_to(np.rad2deg(VIEW_POSE))
        time.sleep(0.5)
        indy.wait_for_move_finish()
        Qcur = np.deg2rad(indy.get_joint_pos())
else:
    Qcur = VIEW_POSE
gscene.show_pose(VIEW_POSE_EXT)

#### 1.1.2  detect bed and add to the scene

In [3]:
bed_center = (2,0,0)
bed_rpy = (0,0,np.pi/2)
COLOR_BED_COL = (0,1,0,0.3)

bed_mat = add_bed(gscene, bed_center, bed_rpy, COLOR_BED_COL)

closet_left, closet_rightup, closet_rightdown = add_closet(
    gscene, closet_center=np.matmul(Rot_rpy(bed_rpy), (-0.75,-1,0))+bed_center, closet_rpy=bed_rpy, 
    COLOR_CLOSET_COL=(0,1,0,0.3))

### 1.2 Make bed cleaning plan

#### 1.2.1 get division-base pose combination data

In [4]:
from pkg.planning.constraint.constraint_common import *
from pkg.planning.constraint.constraint_actor import *
from pkg.planning.constraint.constraint_subject import *

In [5]:
gcheck = GraspChecker(pscene)
mplan.motion_filters = [gcheck]

wp_task, wp_hdl = add_waypoint_task(pscene, "waypoint", WP_DIMS, (0,0,0), (0,0,0), 
                                    parent="floor_ws", color=(0, 0, 1, 0.5))
ccheck = CachedCollisionCheck(gcheck, wp_task, wp_hdl, wayframer)

In [6]:
BED_OFFSET = 0.0
brush_face = pscene.create_binder(bname=TOOL_NAME, gname=TOOL_NAME, _type=SweepFramer, 
                                  point=(0,0,-gscene.NAME_DICT['brush_face'].dims[2]/2-CLEARANCE-BED_OFFSET), 
                                  rpy=(0,0,0))

T_e_brush = brush_face.get_tf_handle(crob.home_dict, from_link=TIP_LINK)
T_brush_e = SE3_inv(T_e_brush)
EE_HEIGHT = round(bed_mat.get_tf(HOME_DICT)[2,3] + bed_mat.dims[2]/2, 5) + T_brush_e[2, 3] - INDY_BASE_OFFSET[2]

In [7]:
ccheck.clear()
div_base_dict, Tsm_keys, surface_div_centers, div_num, (ax_step, ax_swp, ax_pln) = \
                        get_division_dict(bed_mat, brush_face, robot_config, 
                                          plane_val=EE_HEIGHT, tip_dir=None, TOOL_DIM=TOOL_DIM, 
                                          ccheck=ccheck, resolution=0.02, 
                                          sweep_margin=0.1)

Reference Height: 0.552


In [8]:
div_base_dict

defaultdict(<function demo_utils.area_select.<lambda>>,
            {((-0.742, 1.003, -0.6),
              (0.0, 0.0, 0.707, -0.707)): defaultdict(list, {3: [2]}),
             ((-0.738, 1.023, -0.6),
              (0.0, 0.0, 0.707, -0.707)): defaultdict(list, {3: [2]}),
             ((-0.735, 1.063, -0.6),
              (0.0, 0.0, 0.707, -0.707)): defaultdict(list, {3: [2]}),
             ((-0.591, 1.043, -0.6),
              (0.0, 0.0, 0.707, -0.707)): defaultdict(list, {3: [2]}),
             ((-0.45, 1.003, -0.6),
              (0.0, 0.0, 0.707, -0.707)): defaultdict(list, {3: [2, 5]}),
             ((-0.446, 1.023, -0.6),
              (0.0, 0.0, 0.707, -0.707)): defaultdict(list, {3: [2, 5]}),
             ((-0.443, 1.063, -0.6),
              (0.0, 0.0, 0.707, -0.707)): defaultdict(list, {3: [2, 5]}),
             ((-0.424, 0.983, -0.6),
              (0.0, 0.0, 0.707, -0.707)): defaultdict(list, {3: [2, 5]}),
             ((-0.42, 0.943, -0.6),
              (0.0, 0.0, 0.707, -

In [9]:

class TestBaseDivFunc:
    def __init__(self, ppline, floor_ws, surface, WP_DIMS, TOOL_DIM, Q_dict, multiprocess=False,
                 highlight_color=(1, 1, 0, 0.5), tool_dir=1):
        self.ppline, self.floor_ws, self.surface = ppline, floor_ws, surface
        self.WP_DIMS, self.TOOL_DIM, self.Q_dict = WP_DIMS, TOOL_DIM, Q_dict
        self.pscene = self.ppline.pscene
        self.gscene = self.pscene.gscene
        self.multiprocess = multiprocess
        self.tool_dir = tool_dir
        self.highlight_color = highlight_color
        self.pass_count = 0
        self.highlights = []

    def __call__(self, Tsm, swp_centers):
        output = test_base_divs(self.ppline, self.floor_ws, Tsm, self.surface, swp_centers,
                                self.WP_DIMS, self.TOOL_DIM, self.Q_dict, multiprocess=self.multiprocess,
                                tool_dir=self.tool_dir, verbose=True)
#         raise(RuntimeError("test"))
        if output:
            # leave highlight on cleared area
            swp_fin = self.gscene.copy_from(self.gscene.NAME_DICT["sweep"],
                                            new_name="sweep_tested_{}".format(self.pass_count),
                                            color=self.highlight_color)
            swp_fin.dims = (swp_fin.dims[0], swp_fin.dims[1], swp_fin.dims[2] + 0.002)
            self.gscene.update_marker(swp_fin)
            self.highlights.append(swp_fin)
            self.pass_count += 1
        return output

    def clear(self):
        self.pass_count = 0
        for htem in self.highlights:
            self.gscene.remove(htem)
        self.highlights = []


#### 1.2.2 select base poses and generate motions

In [10]:
HOME_POSE_SWEEP = VIEW_POSE_EXT
crob.home_pose = HOME_POSE_SWEEP
crob.home_dict = list2dict(crob.home_pose, gscene.joint_names)
floor_ws = gscene.NAME_DICT["floor_ws"]    
test_fun = TestBaseDivFunc(ppline, floor_ws, bed_mat, WP_DIMS, TOOL_DIM, crob.home_dict)

In [11]:
test_fun.clear()

idx_bases, idc_divs, covered_all, snode_schedule_dict = select_max_cover_bases(
    div_base_dict, Tsm_keys, surface_div_centers, div_num, ax_step,
    test_fun=test_fun)

try: 0 - (0, 0)->(0, 1)
try transition motion
transition motion tried: True
result: 0 - (0, 0)->(0, 1) = success
branching: 0->1 (0.05/3.0 s, steps/err: 13(44.0380573273 ms)/0.00150372693161)
try: 1 - (0, 1)->(1, 1)
try transition motion
transition motion tried: True
result: 1 - (0, 1)->(1, 1) = success
branching: 1->2 (0.1/3.0 s, steps/err: 25(51.7680644989 ms)/0.00197164994995)
try: 2 - (1, 1)->(2, 1)
try constrained motion
singular
constrained motion tried: False
Motion Plan Failure
result: 2 - (1, 1)->(2, 1) = fail
try: 0 - (0, 0)->(0, 1)
try transition motion
transition motion tried: True
result: 0 - (0, 0)->(0, 1) = success
branching: 0->3 (0.19/3.0 s, steps/err: 13(49.7679710388 ms)/0.00140021423615)
try: 3 - (0, 1)->(1, 1)
try transition motion
transition motion tried: True
result: 3 - (0, 1)->(1, 1) = success
branching: 3->4 (0.28/3.0 s, steps/err: 25(84.3868255615 ms)/0.00149326441644)
try: 4 - (1, 1)->(2, 1)
try constrained motion
singular
constrained motion tried: False
Mot

try transition motion
transition motion tried: True
result: 1 - (0, 1)->(1, 1) = success
branching: 1->3 (0.63/3.0 s, steps/err: 25(78.8488388062 ms)/0.00141078280627)
try: 3 - (1, 1)->(2, 1)
try constrained motion
singular
constrained motion tried: False
Motion Plan Failure
result: 3 - (1, 1)->(2, 1) = fail
try: 0 - (0, 0)->(0, 1)
try transition motion
transition motion tried: True
result: 0 - (0, 0)->(0, 1) = success
branching: 0->4 (0.71/3.0 s, steps/err: 13(37.2200012207 ms)/0.00136841187428)
try: 4 - (0, 1)->(1, 1)
try transition motion
transition motion tried: True
result: 4 - (0, 1)->(1, 1) = success
branching: 4->5 (0.76/3.0 s, steps/err: 25(47.5158691406 ms)/0.00159237080504)
try: 5 - (1, 1)->(2, 1)
try constrained motion
singular
constrained motion tried: False
Motion Plan Failure
result: 5 - (1, 1)->(2, 1) = fail
try: 1 - (0, 1)->(1, 1)
try transition motion
transition motion tried: True
result: 1 - (0, 1)->(1, 1) = success
branching: 1->6 (0.87/3.0 s, steps/err: 25(75.00910

result: 0 - (0, 0)->(1, 0) = fail
try: 4 - (0, 1)->(1, 1)
try transition motion
transition motion tried: True
result: 4 - (0, 1)->(1, 1) = success
branching: 4->6 (0.91/3.0 s, steps/err: 25(57.8048229218 ms)/0.0017227929266)
try: 6 - (1, 1)->(2, 1)
try constrained motion
singular
constrained motion tried: False
Motion Plan Failure
result: 6 - (1, 1)->(2, 1) = fail
try: 4 - (0, 1)->(1, 1)
try transition motion
transition motion tried: True
result: 4 - (0, 1)->(1, 1) = success
branching: 4->7 (1.06/3.0 s, steps/err: 25(61.2621307373 ms)/0.00137555439334)
try: 7 - (1, 1)->(2, 1)
try constrained motion
singular
constrained motion tried: False
Motion Plan Failure
result: 7 - (1, 1)->(2, 1) = fail
try: 0 - (0, 0)->(0, 1)
try transition motion
transition motion tried: True
result: 0 - (0, 0)->(0, 1) = success
branching: 0->8 (1.17/3.0 s, steps/err: 12(55.7079315186 ms)/0.00143121221364)
try: 8 - (0, 1)->(1, 1)
try transition motion
transition motion tried: True
result: 8 - (0, 1)->(1, 1) = su

result: 0 - (0, 0)->(1, 0) = fail
try: 1 - (0, 1)->(1, 1)
try transition motion
transition motion tried: True
result: 1 - (0, 1)->(1, 1) = success
branching: 1->3 (0.86/3.0 s, steps/err: 29(46.9779968262 ms)/0.00137366722993)
try: 3 - (1, 1)->(2, 1)
try constrained motion
singular
constrained motion tried: False
Motion Plan Failure
result: 3 - (1, 1)->(2, 1) = fail
try: 1 - (0, 1)->(1, 1)
try transition motion
transition motion tried: True
result: 1 - (0, 1)->(1, 1) = success
branching: 1->4 (0.93/3.0 s, steps/err: 29(50.0030517578 ms)/0.00179905722068)
try: 4 - (1, 1)->(2, 1)
try constrained motion
singular
constrained motion tried: False
Motion Plan Failure
result: 4 - (1, 1)->(2, 1) = fail
try: 1 - (0, 1)->(1, 1)
try transition motion
transition motion tried: True
result: 1 - (0, 1)->(1, 1) = success
branching: 1->5 (1.01/3.0 s, steps/err: 29(71.2220668793 ms)/0.00085862731501)
try: 5 - (1, 1)->(2, 1)
try constrained motion
singular
constrained motion tried: False
Motion Plan Failur

result: 0 - (0, 0)->(1, 0) = fail
try: 0 - (0, 0)->(1, 0)
try transition motion
transition motion tried: False
Motion Plan Failure
result: 0 - (0, 0)->(1, 0) = fail
try: 1 - (0, 1)->(1, 1)
try transition motion
transition motion tried: True
result: 1 - (0, 1)->(1, 1) = success
branching: 1->5 (1.9/3.0 s, steps/err: 26(79.6749591827 ms)/0.00193331104282)
try: 5 - (1, 1)->(2, 1)
try constrained motion
singular
constrained motion tried: False
Motion Plan Failure
result: 5 - (1, 1)->(2, 1) = fail
try: 0 - (0, 0)->(1, 0)
try transition motion
transition motion tried: False
Motion Plan Failure
result: 0 - (0, 0)->(1, 0) = fail
try: 1 - (0, 1)->(1, 1)
try transition motion
transition motion tried: True
result: 1 - (0, 1)->(1, 1) = success
branching: 1->6 (2.34/3.0 s, steps/err: 26(73.4169483185 ms)/0.00201303958985)
try: 6 - (1, 1)->(2, 1)
try constrained motion
singular
constrained motion tried: False
Motion Plan Failure
result: 6 - (1, 1)->(2, 1) = fail
try: 0 - (0, 0)->(1, 0)
try transitio

transition motion tried: True
result: 11 - (0, 1)->(1, 1) = success
branching: 11->12 (0.99/3.0 s, steps/err: 29(92.6430225372 ms)/0.0011332822976)
try: 12 - (1, 1)->(2, 1)
try constrained motion
singular
constrained motion tried: False
Motion Plan Failure
result: 12 - (1, 1)->(2, 1) = fail
try: 4 - (0, 1)->(1, 1)
try transition motion
transition motion tried: True
result: 4 - (0, 1)->(1, 1) = success
branching: 4->13 (1.08/3.0 s, steps/err: 29(72.5190639496 ms)/0.000846876540353)
try: 13 - (1, 1)->(2, 1)
try constrained motion
singular
constrained motion tried: False
Motion Plan Failure
result: 13 - (1, 1)->(2, 1) = fail
try: 0 - (0, 0)->(0, 1)
try transition motion
transition motion tried: True
result: 0 - (0, 0)->(0, 1) = success
branching: 0->14 (1.13/3.0 s, steps/err: 13(39.0479564667 ms)/0.00133680159005)
try: 14 - (0, 1)->(1, 1)
try transition motion
transition motion tried: True
result: 14 - (0, 1)->(1, 1) = success
branching: 14->15 (1.21/3.0 s, steps/err: 29(79.833984375 ms)/

transition motion tried: True
result: 4 - (0, 1)->(1, 1) = success
branching: 4->8 (1.91/3.0 s, steps/err: 31(80.775976181 ms)/0.00141151673625)
try: 8 - (1, 1)->(2, 1)
try constrained motion
singular
constrained motion tried: False
Motion Plan Failure
result: 8 - (1, 1)->(2, 1) = fail
try: 4 - (0, 1)->(1, 1)
try transition motion
transition motion tried: True
result: 4 - (0, 1)->(1, 1) = success
branching: 4->9 (2.01/3.0 s, steps/err: 25(61.9471073151 ms)/0.00144249674784)
try: 9 - (1, 1)->(2, 1)
try constrained motion
singular
constrained motion tried: False
Motion Plan Failure
result: 9 - (1, 1)->(2, 1) = fail
try: 1 - (0, 1)->(1, 1)
try transition motion
transition motion tried: True
result: 1 - (0, 1)->(1, 1) = success
branching: 1->10 (2.12/3.0 s, steps/err: 25(58.0418109894 ms)/0.00123388244116)
try: 10 - (1, 1)->(2, 1)
try constrained motion
singular
constrained motion tried: False
Motion Plan Failure
result: 10 - (1, 1)->(2, 1) = fail
try: 0 - (0, 0)->(0, 1)
try transition mot

result: 0 - (0, 0)->(1, 0) = fail
try: 0 - (0, 0)->(0, 1)
try transition motion
transition motion tried: True
result: 0 - (0, 0)->(0, 1) = success
branching: 0->5 (1.78/3.0 s, steps/err: 10(57.2471618652 ms)/0.00153649860714)
try: 5 - (0, 1)->(1, 1)
try transition motion
transition motion tried: True
result: 5 - (0, 1)->(1, 1) = success
branching: 5->6 (1.84/3.0 s, steps/err: 25(61.0420703888 ms)/0.00216045699499)
try: 6 - (1, 1)->(2, 1)
try constrained motion
singular
constrained motion tried: False
Motion Plan Failure
result: 6 - (1, 1)->(2, 1) = fail
try: 0 - (0, 0)->(0, 1)
try transition motion
transition motion tried: True
result: 0 - (0, 0)->(0, 1) = success
branching: 0->7 (1.92/3.0 s, steps/err: 10(41.4550304413 ms)/0.00179386023406)
try: 7 - (0, 1)->(1, 1)
try transition motion
transition motion tried: True
result: 7 - (0, 1)->(1, 1) = success
branching: 7->8 (1.98/3.0 s, steps/err: 25(58.76994133 ms)/0.00135465478407)
try: 8 - (1, 1)->(2, 1)
try constrained motion
singular
co

transition motion tried: False
Motion Plan Failure
result: 0 - (0, 0)->(1, 0) = fail
======================= terminated 0: max iteration time reached (1633634944/1633634941.29 s) ===============================
try: 0 - (0, 0)->(0, 1)
try transition motion
transition motion tried: True
result: 0 - (0, 0)->(0, 1) = success
branching: 0->1 (0.07/3.0 s, steps/err: 10(72.0629692078 ms)/0.00136617816576)
try: 1 - (0, 1)->(1, 1)
try transition motion
transition motion tried: True
result: 1 - (0, 1)->(1, 1) = success
branching: 1->2 (0.15/3.0 s, steps/err: 29(70.9748268127 ms)/0.00144498623493)
try: 2 - (1, 1)->(2, 1)
try constrained motion
singular
constrained motion tried: False
Motion Plan Failure
result: 2 - (1, 1)->(2, 1) = fail
try: 1 - (0, 1)->(1, 1)
try transition motion
transition motion tried: True
result: 1 - (0, 1)->(1, 1) = success
branching: 1->3 (0.23/3.0 s, steps/err: 29(67.2481060028 ms)/0.00107199195701)
try: 3 - (1, 1)->(2, 1)
try constrained motion
singular
constrained mot

IndexError: string index out of range

In [ ]:
snode = tplan.snode_dict[2]

In [ ]:
snode = tplan.snode_dict[3]

In [ ]:
gscene.show_pose(snode.state.Q)

In [ ]:
np.round(np.rad2deg(snode.state.Q), 3)[6:]

#### 1.2.3 refine motions

In [13]:
HOME_POSE_MOVE = VIEW_POSE
snode_schedule_list, scene_args_list, scene_kwargs_list = refine_order_plan(
    ppline, snode_schedule_dict, idx_bases, idc_divs, Qcur, 
    floor_ws, wayframer, bed_mat, Tsm_keys, surface_div_centers,  
    WP_DIMS, TOOL_DIM, ROBOT_NAME, MOBILE_NAME, HOME_POSE_MOVE)

Goal reached
Goal reached
Goal reached
Goal reached
Goal reached
Goal reached
Goal reached
Goal reached
Goal reached
Goal reached
Goal reached
Goal reached
Goal reached


### 1.3 Execute bed cleaning sequence

In [14]:
# indy.traj_vel = 3

In [15]:
swp_fin_list = []
for i_s, (snode_schedule, sargs, skwargs) in enumerate(zip(snode_schedule_list, scene_args_list, scene_kwargs_list)):
    set_base_sweep(*sargs, **skwargs)
    for snode_pre, snode_nxt in zip(snode_schedule[:-1], snode_schedule[1:]):
        from_state = snode_pre.state
        to_state = snode_nxt.state
        subjects, ok = pscene.get_changing_subjects(from_state, to_state)

        if len(subjects) ==0 or subjects[0] == "sweep":
            if CONNECT_INDY:
                ppline.execute_schedule([snode_pre, snode_nxt], one_by_one=True)
                with indy:
                    time.sleep(0.5)
                    indy.wait_for_move_finish()
            else:
                ppline.play_schedule([snode_pre, snode_nxt], period=0.002)

        elif subjects[0] == "waypoints":
            Tbw0 = wayframer.get_tf_handle(list2dict(from_state.Q, gscene.joint_names))
            Tbw1 = wayframer.get_tf_handle(list2dict(to_state.Q, gscene.joint_names))
            Tw0w1 = np.matmul(SE3_inv(Tbw0), Tbw1)
            cur_xyzw, tar_xyzw_rd, tar_xyzw = get_relative_mobile_command(Tw0w1, CONNECT_MOBILE)
            ref_xyzw = cur_xyzw
            delta_xyzw = np.subtract(tar_xyzw, ref_xyzw)
            tar_xyzw = delta_xyzw + ref_xyzw
            cur_xyzw = move_mobile_robot(sock_mobile, cur_xyzw, tar_xyzw, tar_xyzw, MOBILE_IP, CONNECT_MOBILE, move_direct=True)
            if not CONNECT_MOBILE:
                gscene.show_motion(snode_nxt.traj)
            gscene.show_pose(to_state.Q)
    # leave highlight on cleared area
    swp_fin = gscene.copy_from(gscene.NAME_DICT["sweep"], new_name="sweep_fin_{}".format(i_s), color=(0,0,1,0.5))
    swp_fin.dims = (swp_fin.dims[0], swp_fin.dims[1], swp_fin.dims[2]+0.002)
    gscene.update_marker(swp_fin)
    swp_fin_list.append(swp_fin)

curre  xyzw: [0 0 0 1]
ready  xyzw: [ 0.    0.   -0.41  0.58]
target xyzw: [ 0.   -0.47 -0.41  0.58]
move to: [ 0.   -0.47 -0.41  0.58]
(0, 1)->(1, 1)
(1, 1)->(2, 1)
(2, 1)->(2, 1)
curre  xyzw: [0 0 0 1]
ready  xyzw: [-0.22  0.   -0.    1.  ]
target xyzw: [ 0.28 -0.   -0.    1.  ]
move to: [ 0.28 -0.   -0.    1.  ]
(0, 1)->(1, 1)
(1, 1)->(2, 1)
(2, 1)->(2, 1)
curre  xyzw: [0 0 0 1]
ready  xyzw: [-0.5  -0.29  0.    1.  ]
target xyzw: [ 0.   -0.29  0.    1.  ]
move to: [ 0.   -0.29  0.    1.  ]
(0, 1)->(1, 1)
(1, 1)->(2, 1)
(2, 1)->(2, 1)
curre  xyzw: [0 0 0 1]
ready  xyzw: [-0.79  0.   -0.    1.  ]
target xyzw: [-0.29  0.   -0.    1.  ]
move to: [-0.29  0.   -0.    1.  ]
(0, 1)->(1, 1)
(1, 1)->(2, 1)
(2, 1)->(2, 1)
curre  xyzw: [0 0 0 1]
ready  xyzw: [-0.5  -0.29  0.    1.  ]
target xyzw: [ 0.   -0.29  0.    1.  ]
move to: [ 0.   -0.29  0.    1.  ]
(0, 1)->(1, 1)
(1, 1)->(2, 1)
(2, 1)->(2, 1)
curre  xyzw: [0 0 0 1]
ready  xyzw: [-0.5   0.88 -0.    1.  ]
target xyzw: [ 0.    0.88 -0.    

In [16]:
np.round(gscene.get_tf(TIP_LINK, snode_schedule[2].state.Q, from_link=ROBOT_BASE), 3)

array([[-0.001, -0.   , -1.   , -0.59 ],
       [-0.001, -1.   ,  0.   , -0.1  ],
       [-1.   ,  0.001,  0.001,  0.652],
       [ 0.   ,  0.   ,  0.   ,  1.   ]])

In [ ]:
# test_fun.clear()
# for swp_fin in swp_fin_list:
#     gscene.remove(swp_fin)
# swp_fin_list = []
# pscene.clear_subjects()
# for child in copy.copy(bed_mat.children):
#     gscene.remove(gscene.NAME_DICT[child])


## 2. Closet cleaning

### 2.2. Make closet cleaning plan

#### 2.2.1 make plans

In [ ]:
from pkg.planning.constraint.constraint_common import *
from pkg.planning.constraint.constraint_actor import *
from pkg.planning.constraint.constraint_subject import *

In [ ]:
wp_task, wp_hdl = add_waypoint_task(pscene, "waypoint", WP_DIMS, (0,0,0), (0,0,0), 
                                    parent="floor_ws", color=(0, 0, 1, 0.5))
brush_face = pscene.create_binder(bname=TOOL_NAME, gname=TOOL_NAME, _type=SweepFramer, 
                                  point=(-gscene.NAME_DICT['brush_face'].dims[0]/2-CLEARANCE,0,0), 
                                  rpy=(0,np.pi/2*1,0))
HOME_POSE_SWEEP = VIEW_POSE_EXT
crob.home_pose = HOME_POSE_SWEEP
crob.home_dict = list2dict(crob.home_pose, gscene.joint_names)
floor_ws = gscene.NAME_DICT["floor_ws"]    


In [ ]:
ccheck.clear()
div_base_dict, Tsm_keys, surface_div_centers, div_num, (ax_step, ax_swp, ax_pln) = \
                        get_division_dict(closet_left, brush_face, robot_config, 
                                          plane_val=None, tip_dir="up", TOOL_DIM=TOOL_DIM, 
                                          ccheck=ccheck, resolution=0.02)
test_fun_cl = TestBaseDivFunc(ppline, floor_ws, closet_left, WP_DIMS, TOOL_DIM, crob.home_dict, tool_dir=1)
test_fun_cl.clear()

idx_bases, idc_divs, covered_all, snode_schedule_dict = select_max_cover_bases(
    div_base_dict, Tsm_keys, surface_div_centers, div_num, ax_step, 
    test_fun=test_fun_cl)
HOME_POSE_MOVE = VIEW_POSE
snode_schedule_list_up, scene_args_list_up, scene_kwargs_list_up = refine_order_plan(
    ppline, snode_schedule_dict, idx_bases, idc_divs, Qcur, 
    floor_ws, wayframer, closet_left, Tsm_keys, surface_div_centers,  
    WP_DIMS, TOOL_DIM, ROBOT_NAME, MOBILE_NAME, HOME_POSE_MOVE)

In [ ]:
ccheck.clear()
div_base_dict, Tsm_keys, surface_div_centers, div_num, (ax_step, ax_swp, ax_pln) = \
                        get_division_dict(closet_left, brush_face, robot_config, 
                                          plane_val=None, tip_dir="down", TOOL_DIM=TOOL_DIM, 
                                          ccheck=ccheck, resolution=0.02)
test_fun_cl = TestBaseDivFunc(ppline, floor_ws, closet_left, WP_DIMS, TOOL_DIM, crob.home_dict, tool_dir=-1)
test_fun_cl.clear()

idx_bases, idc_divs, covered_all, snode_schedule_dict = select_max_cover_bases(
    div_base_dict, Tsm_keys, surface_div_centers, div_num, ax_step, 
    test_fun=test_fun_cl)
HOME_POSE_MOVE = VIEW_POSE
snode_schedule_list_down, scene_args_list_down, scene_kwargs_list_down = refine_order_plan(
    ppline, snode_schedule_dict, idx_bases, idc_divs, Qcur, 
    floor_ws, wayframer, closet_left, Tsm_keys, surface_div_centers,  
    WP_DIMS, TOOL_DIM, ROBOT_NAME, MOBILE_NAME, HOME_POSE_MOVE)

In [18]:
snode_schedule_list = snode_schedule_list_up + snode_schedule_list_down
scene_args_list = scene_args_list_up + scene_args_list_down
scene_kwargs_list = scene_kwargs_list_up + scene_kwargs_list_down

### 2.3 Execute bed cleaning sequence

In [19]:
# indy.traj_vel = 3

In [20]:
swp_fin_list = []
for i_s, (snode_schedule, sargs, skwargs) in enumerate(zip(snode_schedule_list, scene_args_list, scene_kwargs_list)):
    set_base_sweep(*sargs, **skwargs)
    for snode_pre, snode_nxt in zip(snode_schedule[:-1], snode_schedule[1:]):
        from_state = snode_pre.state
        to_state = snode_nxt.state
        subjects, ok = pscene.get_changing_subjects(from_state, to_state)

        if len(subjects) ==0 or subjects[0] == "sweep":
            if CONNECT_INDY:
                ppline.execute_schedule([snode_pre, snode_nxt], one_by_one=True)
                with indy:
                    time.sleep(0.5)
                    indy.wait_for_move_finish()
            else:
                ppline.play_schedule([snode_pre, snode_nxt])

        elif subjects[0] == "waypoints":
            Tbw0 = wayframer.get_tf_handle(list2dict(from_state.Q, gscene.joint_names))
            Tbw1 = wayframer.get_tf_handle(list2dict(to_state.Q, gscene.joint_names))
            Tw0w1 = np.matmul(SE3_inv(Tbw0), Tbw1)
            cur_xyzw, tar_xyzw_rd, tar_xyzw = get_relative_mobile_command(Tw0w1, CONNECT_MOBILE)
            ref_xyzw = cur_xyzw
            delta_xyzw = np.subtract(tar_xyzw, ref_xyzw)
            tar_xyzw = delta_xyzw + ref_xyzw
            cur_xyzw = move_mobile_robot(sock_mobile, cur_xyzw, tar_xyzw, tar_xyzw, MOBILE_IP, CONNECT_MOBILE, move_direct=True)
            if not CONNECT_MOBILE:
                gscene.show_motion(snode_nxt.traj)
            gscene.show_pose(to_state.Q)
    # leave highlight on cleared area
    swp_fin = gscene.copy_from(gscene.NAME_DICT["sweep"], new_name="sweep_fin_{}".format(i_s), color=(0,0,1,0.5))
    swp_fin.dims = (swp_fin.dims[0], swp_fin.dims[1], swp_fin.dims[2]+0.002)
    gscene.update_marker(swp_fin)
    swp_fin_list.append(swp_fin)

curre  xyzw: [0 0 0 1]
ready  xyzw: [-0.48 -2.9  -0.63  0.11]
target xyzw: [-0.4  -3.07 -0.63  0.11]
move to: [-0.4  -3.07 -0.63  0.11]
move to: [-0.4  -3.07 -0.63  0.11]
(0, 1)->(1, 1)
(1, 1)->(2, 1)
(2, 1)->(2, 1)
curre  xyzw: [0 0 0 1]
ready  xyzw: [-0.49 -2.82 -0.63  0.11]
target xyzw: [-0.41 -3.   -0.63  0.11]
move to: [-0.41 -3.   -0.63  0.11]
move to: [-0.41 -3.   -0.63  0.11]
(0, 1)->(1, 1)
(1, 1)->(2, 1)
(2, 1)->(2, 1)


In [24]:
test_fun_cl.clear()
for swp_fin in swp_fin_list:
    gscene.remove(swp_fin)
swp_fin_list = []
pscene.clear_subjects()
for child in copy.copy(closet_left.children):
    gscene.remove(gscene.NAME_DICT[child])

## 3. Return to home